In [ ]:
# module loading
from keras.src.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose
from keras.src.layers import Concatenate, Input
from keras.src.models import Model

In [ ]:
# normal convolution block
def conv_block(x, num_filters, kernel_size, padding="same", act=True):
    x = Conv2D(num_filters, kernel_size, padding=padding, use_bias=False)(x)
    x = BatchNormalization()(x)
    if act == True:
        x = Activation("relu")(x)
    return x

In [ ]:
# multires block with 3 skips
def multires_block(x, num_filters, alpha=1.67):
    W = num_filters * alpha

    x0 = x

    # 3 conv blocks in a multires block
    x1 = conv_block(x0, int(W*0.167), 3)
    x2 = conv_block(x1, int(W*0.333), 3)
    x3 = conv_block(x2, int(W*0.5), 3)
    xc = Concatenate()([x1, x2, x3])
    xc = BatchNormalization()(xc)

    nf = int(W*0.167) + int(W*0.333) + int(W*0.5)
    sc = conv_block(x0, nf, 1, act=False)

    x = Activation("relu")(xc + sc)
    x = BatchNormalization()(x)
    return x

In [ ]:
# skip connections
def res_path(x, num_filters, length):
    for i in range(length):
        x0 = x
        x1 = conv_block(x0, num_filters, 3, act=False)
        sc = conv_block(x0, num_filters, 1, act=False)
        x = Activation("relu")(x1 + sc)
        x = BatchNormalization()(x)
    return x


In [ ]:
# downstream
def encoder_block(x, num_filters, length):
    x = multires_block(x, num_filters)
    s = res_path(x, num_filters, length)
    p = MaxPooling2D((2, 2))(x)
    return s, p

In [ ]:
# upstream
def decoder_block(x, skip, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(x)
    x = Concatenate()([x, skip])
    x = multires_block(x, num_filters)
    return x

In [ ]:
# model compile
def build_multiresunet(shape):
    """ Input """
    inputs = Input(shape)

    """ Encoder """
    p0 = inputs
    s1, p1 = encoder_block(p0, 32, 4)
    s2, p2 = encoder_block(p1, 64, 3)
    s3, p3 = encoder_block(p2, 128, 2)
    s4, p4 = encoder_block(p3, 256, 1)

    """ Bridge """
    b1 = multires_block(p4, 512)

    """ Decoder """
    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs, name="multires-unet")

    return model

In [ ]:
if __name__ == "__main__":
    shape = (256, 256, 3)
    model = build_multiresunet(shape)
    model.summary()

In [9]:
# module loading for preprocessing
import os
import json
import numpy as np
from PIL import Image

In [10]:
# paths to specified directories
gtFine_dir = 'content/gtFine/'
leftImg8bit_dir = 'content/leftImg8bit'

In [11]:
# extract unique colors from ground truth labeled images and save the mappings to a JSON file
def extract_unique_colors(gt_fine_dir, split='train'):

    # initialize empty dict (stores unique colors with corresponding unique index)
    unique_colors = {}
    labels_dir = os.path.join(gt_fine_dir, split)
    cities = os.listdir(labels_dir)

    for city in cities:
        city_dir = os.path.join(labels_dir, city)
        label_files = [f for f in os.listdir(city_dir) if f.endswith('_gtFine_color.png')]

        for file in label_files:
            label_path = os.path.join(city_dir, file)
            label_image = Image.open(label_path).convert('RGB')
            label_array = np.array(label_image)
            colors = np.unique(label_array.reshape(-1, label_array.shape[2]), axis=0)
            for color in colors:
                unique_colors[tuple(color)] = len(unique_colors)
            print('file '+ label_path+' extracted')

    # Cache the class2color and color2class mappings
    with open('class2color.json', 'w') as f:
        json.dump({str(k): v for k, v in unique_colors.items()}, f)

    return unique_colors

In [12]:
# load the saved color-to-class mappings from 'class2color.json' and return both class-to-color and color-to-class mappings
def load_class2color_mapping():
    with open('class2color.json', 'r') as f:
        class2color = json.load(f)
    color2class = {tuple(map(int, k.strip("()").split(','))): v for k, v in class2color.items()}
    return color2class

In [13]:
# converting labeled image into a class index map using the color2class mapping
def color_to_class(label, color2class):
    label_class = np.zeros((label.shape[0], label.shape[1]), dtype=np.int64)
    for color, class_idx in color2class.items():
        mask = np.all(label == np.array(color), axis=-1)
        label_class[mask] = class_idx
    return label_class

In [ ]:
# creating dataset class to handle semantic segmentations
